In [1]:
import tensorflow as tf

from gensim.models.keyedvectors import KeyedVectors
from konlpy.tag import Mecab

from googletrans import Translator

import time
import numpy as np

import os
import sys
import urllib.request
import requests
import datetime
import pickle
import json

In [2]:
with open('./data/ko_noun_dict.pkl', "rb") as f:
    ko_dict = pickle.load(f)
    
with open('./data/en_noun_dict.pkl', "rb") as f:
    en_dict = pickle.load(f)

In [3]:
def get_noun_data(ko_dict, en_dict):
    for i in range(len(ko_dict)):
        ko_vector = list(ko_dict.values())[i]
        en_vector = list(en_dict.values())[i]
        
#         yield (ko_word, ko_vector, en_word, en_vector)
        yield (ko_vector, en_vector)
    
def get_noun_data_2(ko_vec, en_vec):
    for i in range(len(ko_vec)):
        ko_vector = ko_vec[i]
        en_vector = en_vec[i]
        
        yield ko_vector, en_vector

In [4]:
dataset = tf.data.Dataset.from_generator(get_noun_data_2, 
                              (tf.float64, tf.float64),
                              (tf.TensorShape([300]), tf.TensorShape([300])),
                               args=(list(ko_dict.values()), list(en_dict.values())))

In [5]:
from models.transformer import * 

In [8]:
sample_enc = Encoder(num_layers=1, d_model=1, num_heads=1, dff=512, input_vocab_size=0, maximum_position_encoding=0)

In [9]:
temp_input = tf.random.uniform((64, 300), dtype=tf.float32, minval=0, maxval=1)
sample_encoder_output = sample_enc(temp_input, training=False, mask=None)
print(sample_encoder_output.shape)

Model Input shape (64, 300, 1)
Scaled_attention Shape :  (64, 1, 300, 1)
Scaled_attention Shape :  (64, 300, 1, 1)
Concat attention Shape : (64, 300, 1)
(64, 300, 1)


In [28]:
encoder = Encoder(num_layers=1, d_model=1, num_heads=1, dff=512, input_vocab_size=0, maximum_position_encoding=0)

In [17]:
EPOCHS = 20

num_layers = 1
d_model = 1
dff = 512
num_head = 1
dropout_rate = 0.1

In [29]:
loss_object = tf.keras.losses.Huber()

def loss_function(real, pred):
    loss = loss_object(real, pred)
    
    return loss


In [30]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
optimizer = tf.keras.optimizers.Adam()

In [35]:
@tf.function()
def train_step(inp, real):
    
    with tf.GradientTape() as tape:
        output = encoder(inp, training=True, mask=None)
        loss = loss_function(real, output)
        
    gradients = tape.gradient(loss, encoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, encoder.trainable_variables))
    
    train_loss(loss)

In [36]:
for epoch in range(EPOCHS):
    tic = time.time()
    
    train_loss.reset_states()
    
    for (batch, (inp, real)) in enumerate(dataset):
        train_step(inp, real)
        
        if batch % 50 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result() : .4f}')
                  
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'Saving checkpoint for epoch {epoch + 1} at {ckpt_save_path}')

    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Model Input shape (300, 1)
Scaled_attention Shape :  (300, 1, 1, 1)
Scaled_attention Shape :  (300, 1, 1, 1)
Concat attention Shape : (300, 1, 1)
Model Input shape (300, 1)
Scaled_attention Shape :  (300, 1, 1, 1)
Scaled_attention Shape :  (300, 1, 1, 1)
Concat attention Shape : (300, 1, 1)
Epoch 1 Batch 0 Loss  0.0262
Epoch 1 Batch 50 Loss  0.0368
Epoch 1 Batch 100 Loss  0.0375
Epoch 1 Batch 150 Loss  0.0391
Epoch 1 Batch 200 Loss  0.0393
Epoch 1 Batch 250 Loss  0.0396
Epoch 1 Batch 300 Loss  0.0398
Epoch 1 Batch 350 Loss  0.0394
Epoch 1 Batch 400 Loss  0.0395
Epoch 1 Batch 450 Loss  0.0397
Epoch 1 Batch 500 Loss  0.0398
Epoch 1 Batch 550 Loss  0.0399


UnknownError: IndexError: index 576 is out of bounds for axis 0 with size 576
Traceback (most recent call last):

  File "/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 236, in __call__
    ret = func(*args)

  File "/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/data/ops/dataset_ops.py", line 789, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-3-e1fa3de85435>", line 12, in get_noun_data_2
    en_vector = en_vec[i]

IndexError: index 576 is out of bounds for axis 0 with size 576


	 [[{{node PyFunc}}]]

In [27]:
for (batch, (inp, real)) in enumerate(dataset):
    print(sample_enc(inp, training=False, mask=None))
    break

Model Input shape (300, 1)
Scaled_attention Shape :  (300, 1, 1, 1)
Scaled_attention Shape :  (300, 1, 1, 1)
Concat attention Shape : (300, 1, 1)
tf.Tensor(
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]], shape=(300, 300, 1), dtype=float32)
